# Analysis of Tesla PAB

This jupyter notebook presents the data along with the calculations for added transparency. Section 1 will read the data and create functions and plots, while [Section 2](#results) will show the data and plots.

## 1 Preparation

### 1.1 Python Packages

Pandas will be used for manipulating the excel table data, while seaborn will help with displaying the plots.

In [1]:
import os
import pandas as pd
import seaborn as sns
sns.set()
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
# https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916 

Next we read the excel sheets representing the front, and side data results from the MLCT program. We'll extract just the columns we need with the double brackets, then display the first 5 rows of the front dataframe (front_df).

In [2]:
front_df = pd.read_excel(r'data/front_data.xlsx', index_col='completed')[['Frame', 'sample', 'Raw Score']]
side_df = pd.read_excel(r'data/side_data.xlsx', index_col='completed')[['Frame', 'sample', 'Raw Score']]
front_df.index.name = 'Date'
side_df.index.name = 'Date'
front_df.head()

,Frame,sample,Raw Score
Date,,,
2021-01-27 20:17:24,16,CS656175421015AAXZ,567.452957
2021-01-27 20:17:24,40,CS656175421015AAXZ,436.407660
2021-01-27 20:17:24,25,CS656175421015AAXZ,644.879494
2021-01-27 20:17:24,13,CS656175421015AAXZ,491.289790
2021-01-27 20:17:24,19,CS656175421015AAXZ,883.335840


### 1.2 $\bar{X}$ and MR Chart

We'll create a function that takes the dataframe, frame number (in milliseconds), and a time period to group data by. The function will provide an $\bar{X}$ and MR plot. (Mean and Moving Range).

In [3]:
def x_bar_and_mr(df, frame, time_period):
    this_df = df.copy()
    df1 = this_df[this_df['Frame'] == frame]
    
    if time_period.lower() == 'all':
        scores = df1.copy()
    else:
        scores = df1.resample(time_period).mean().dropna()[['Raw Score']]

    ranges = scores.copy()
    ranges = ranges.shift(1)
    ranges['Prev'] = scores['Raw Score']
    ranges['Range'] = ranges['Raw Score'] - ranges['Prev']

    # Define plot
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 10), sharex=True)
    fig.suptitle(r'$\bar{{{0}}}$ & MR chart for {1} ms'.format('X', frame), fontsize=20)
    # X Bar chart
    sns.lineplot(ax=ax1, y='Raw Score', x=scores.index, marker='o', data=scores, color='k')
    mean = scores['Raw Score'].mean()
    cl = 3*scores['Raw Score'].std()
    ucl = mean + cl
    lcl = mean - cl
    ax1.axhline(scores['Raw Score'].mean(), color='b')
    ax1.axhline(ucl, color='r', linestyle='dashed')
    ax1.axhline(lcl, color='r', linestyle='dashed')
    ax1.set_ylabel(r'$\bar{{{0}}}$'.format('X'))
    ax1.set_xlabel('')

    # Moving Range chart
    sns.lineplot(ax=ax2, y='Range', x=ranges.index, marker='o', data=ranges, color='k')
    mean = ranges['Range'].mean()
    cl = 3*ranges['Range'].std()
    ucl = mean + cl
    lcl = mean - cl
    ax2.axhline(ranges['Range'].mean(), color='b')
    ax2.axhline(ucl, color='r', linestyle='dashed')
    ax2.axhline(lcl, color='r', linestyle='dashed')
    ax2.set_ylabel('MR')
    ax2.set_xlabel('Date')
    plt.savefig('xbarmr.png')
    plt.show()

This snippet provides a simple user interface to select the frame and time to group the data by. It is defined here, but implemented in section 2.

In [6]:
dialog_xbmr = widgets.Output()
charts_xbmr = widgets.Output()

frames = front_df.sort_values(by='Frame')['Frame'].unique().tolist()
lbl_frame = widgets.Label('Frame Number (ms)')
drop_down_frame = widgets.Dropdown(options = frames)
lbl_time = widgets.Label('Time Period for Resampling')
drop_down_time = widgets.Dropdown(options = [('All', 'all'), ('Day', 'D'), ('Week', 'W'), ('Month', 'M')])
btn = widgets.Button(description='Show XBar and MR chart')
label_frames = widgets.VBox([lbl_frame, drop_down_frame])
label_dropdowns = widgets.VBox([lbl_time, drop_down_time])
labels = widgets.HBox([label_frames, label_dropdowns])

def show_chart(b):
    with charts_xbmr:
        charts_xbmr.clear_output()
        x_bar_and_mr(front_df, drop_down_frame.value, drop_down_time.value)
            
with dialog_xbmr:
    display(labels)
    display(btn)
    
    btn.on_click(show_chart)

<a id='results'></a>
## 2 Results 

### 2.1 $\bar{X}$ and MR Chart

In [7]:
display(dialog_xbmr)
display(charts_xbmr)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(children=(VBox(children=(Label(va…

Output()